<a href="https://colab.research.google.com/github/aarohishaiva/Phonepay_project_2/blob/main/Project_2_Phone_pe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages
!pip install mysql-connector-python streamlit plotly pillow streamlit-option-menu
!pip install sqlalchemy

# Import libraries
import pandas as pd
import streamlit as st
import plotly.express as px
import os
import json
from streamlit_option_menu import option_menu
from PIL import Image


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.4/823.4 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [2]:
!git clone https://github.com/PhonePe/pulse.git

Cloning into 'pulse'...
remote: Enumerating objects: 16381, done.
remote: Counting objects: 100% (1179/1179), done.
remote: Compressing objects: 100% (1001/1001), done.
remote: Total 16381 (delta 159), reused 1114 (delta 157), pack-reused 15202 (from 1)
Receiving objects: 100% (16381/16381), 23.67 MiB | 13.51 MiB/s, done.
Resolving deltas: 100% (7901/7901), done.
Updating files: 100% (7921/7921), done.


In [3]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import os
import json


In [6]:
path1 = "/content/pulse/data/aggregated/transaction/country/india/state/"
agg_trans_list = os.listdir(path1)

columns1 = {'State': [], 'Year': [], 'Quarter': [], 'Transaction_type': [], 'Transaction_count': [],
            'Transaction_amount': []}
for state in agg_trans_list:
    all_state = path1 + state + "/"
    agg_year_list = os.listdir(all_state)

    for year in agg_year_list:
        all_year = all_state + year + "/"
        agg_file_list = os.listdir(all_year)

        for file in agg_file_list:
            cur_file = all_year + file
            data = open(cur_file, 'r')
            A = json.load(data)

            for i in A['data']['transactionData']:
                name = i['name']
                count = i['paymentInstruments'][0]['count']
                amount = i['paymentInstruments'][0]['amount']
                columns1['Transaction_type'].append(name)
                columns1['Transaction_count'].append(count)
                columns1['Transaction_amount'].append(amount)
                columns1['State'].append(state)
                columns1['Year'].append(year)
                columns1['Quarter'].append(int(file.strip('.json')))

df_agg_trans = pd.DataFrame(columns1)

In [8]:
df_agg_trans.shape

(4494, 6)

In [10]:
df_agg_trans

,State,Year,Quarter,Transaction_type,Transaction_count,Transaction_amount
0,maharashtra,2020,1,Merchant payments,100337738,3.520013e+10
1,maharashtra,2020,1,Peer-to-peer payments,87067410,2.625412e+11
2,maharashtra,2020,1,Recharge & bill payments,50199498,2.160357e+10
3,maharashtra,2020,1,Financial Services,123476,9.136368e+07
4,maharashtra,2020,1,Others,1215959,5.589320e+08
...,...,...,...,...,...,...
4489,chandigarh,2019,4,Peer-to-peer payments,927850,3.586764e+09
4490,chandigarh,2019,4,Merchant payments,561630,3.186925e+08
4491,chandigarh,2019,4,Recharge & bill payments,338203,2.376956e+08
4492,chandigarh,2019,4,Financial Services,1911,6.542987e+05


In [11]:
path2 = "/content/pulse/data/aggregated/user/country/india/state/"

agg_user_list = os.listdir(path2)

columns2 = {'State': [], 'Year': [], 'Quarter': [], 'Brands': [], 'Count': [],
            'Percentage': []}
for state in agg_user_list:
    cur_state = path2 + state + "/"
    agg_year_list = os.listdir(cur_state)

    for year in agg_year_list:
        cur_year = cur_state + year + "/"
        agg_file_list = os.listdir(cur_year)

        for file in agg_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            B = json.load(data)
            try:
                for i in B["data"]["usersByDevice"]:
                    brand_name = i["brand"]
                    counts = i["count"]
                    percents = i["percentage"]
                    columns2["Brands"].append(brand_name)
                    columns2["Count"].append(counts)
                    columns2["Percentage"].append(percents)
                    columns2["State"].append(state)
                    columns2["Year"].append(year)
                    columns2["Quarter"].append(int(file.strip('.json')))
            except:
                pass
df_agg_user = pd.DataFrame(columns2)
df_agg_user

,State,Year,Quarter,Brands,Count,Percentage
0,maharashtra,2020,1,Xiaomi,6677147,0.255161
1,maharashtra,2020,1,Samsung,5772343,0.220585
2,maharashtra,2020,1,Vivo,4396537,0.168009
3,maharashtra,2020,1,Oppo,3377366,0.129063
4,maharashtra,2020,1,Realme,1031169,0.039405
...,...,...,...,...,...,...
6727,chandigarh,2019,4,OnePlus,8750,0.030215
6728,chandigarh,2019,4,Motorola,6953,0.024010
6729,chandigarh,2019,4,Huawei,4559,0.015743
6730,chandigarh,2019,4,Lenovo,3777,0.013043


In [13]:
path3 = "/content/pulse/data/map/transaction/hover/country/india/state/"  # Add a trailing slash

map_trans_list = os.listdir(path3)

columns3 = {'State': [], 'Year': [], 'Quarter': [], 'District': [], 'Count': [],
            'Amount': []}

for state in map_trans_list:
    cur_state = path3 + state + "/"  # Now correctly constructs the path with the trailing slash
    map_year_list = os.listdir(cur_state)

    for year in map_year_list:
        cur_year = cur_state + year + "/"
        map_file_list = os.listdir(cur_year)

        for file in map_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            C = json.load(data)

            for i in C["data"]["hoverDataList"]:
                district = i["name"]
                count = i["metric"][0]["count"]
                amount = i["metric"][0]["amount"]
                columns3["District"].append(district)
                columns3["Count"].append(count)
                columns3["Amount"].append(amount)
                columns3['State'].append(state)
                columns3['Year'].append(year)
                columns3['Quarter'].append(int(file.strip('.json')))

df_map_trans = pd.DataFrame(columns3)
df_map_trans

,State,Year,Quarter,District,Count,Amount
0,maharashtra,2020,1,jalgaon district,2925407,4.531030e+09
1,maharashtra,2020,1,latur district,2119578,3.427202e+09
2,maharashtra,2020,1,thane district,15699451,2.255105e+10
3,maharashtra,2020,1,bhandara district,687372,1.008982e+09
4,maharashtra,2020,1,nandurbar district,509151,9.110139e+08
...,...,...,...,...,...,...
18291,chandigarh,2024,1,chandigarh district,13155173,1.584218e+10
18292,chandigarh,2019,1,chandigarh district,1274476,2.396032e+09
18293,chandigarh,2019,2,chandigarh district,1252470,2.693116e+09
18294,chandigarh,2019,3,chandigarh district,1506543,3.269682e+09


In [14]:
path4 = "/content/pulse/data/map/user/hover/country/india/state/" # Add trailing slash to ensure correct path construction

map_user_list = os.listdir(path4)

columns4 = {"State": [], "Year": [], "Quarter": [], "District": [],
            "RegisteredUser": [], "AppOpens": []}

for state in map_user_list:
    cur_state = path4 + state + "/"
    map_year_list = os.listdir(cur_state)

    for year in map_year_list:
        cur_year = cur_state + year + "/"
        map_file_list = os.listdir(cur_year)

        for file in map_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            D = json.load(data)

            for i in D["data"]["hoverData"].items():
                district = i[0]
                registereduser = i[1]["registeredUsers"]
                appOpens = i[1]['appOpens']
                columns4["District"].append(district)
                columns4["RegisteredUser"].append(registereduser)
                columns4["AppOpens"].append(appOpens)
                columns4['State'].append(state)
                columns4['Year'].append(year)
                columns4['Quarter'].append(int(file.strip('.json')))

df_map_user = pd.DataFrame(columns4)
df_map_user

,State,Year,Quarter,District,RegisteredUser,AppOpens
0,maharashtra,2020,1,jalgaon district,605036,12727925
1,maharashtra,2020,1,latur district,398366,9141940
2,maharashtra,2020,1,thane district,2878073,53154761
3,maharashtra,2020,1,bhandara district,165915,3071689
4,maharashtra,2020,1,nandurbar district,128739,2392415
...,...,...,...,...,...,...
18295,chandigarh,2024,1,chandigarh district,754849,2680802
18296,chandigarh,2019,1,chandigarh district,209175,0
18297,chandigarh,2019,2,chandigarh district,240439,1220176
18298,chandigarh,2019,3,chandigarh district,267337,4625196


In [15]:
path5 = "/content/pulse/data/top/transaction/country/india/state/"

top_trans_list = os.listdir(path5)
columns5 = {'State': [], 'Year': [], 'Quarter': [], 'Pincode': [], 'Transaction_count': [],
            'Transaction_amount': []}

for state in top_trans_list:
    cur_state = path5 + state + "/"
    top_year_list = os.listdir(cur_state)

    for year in top_year_list:
        cur_year = cur_state + year + "/"
        top_file_list = os.listdir(cur_year)

        for file in top_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            E = json.load(data)

            for i in E['data']['pincodes']:
                name = i['entityName']
                count = i['metric']['count']
                amount = i['metric']['amount']
                columns5['Pincode'].append(name)
                columns5['Transaction_count'].append(count)
                columns5['Transaction_amount'].append(amount)
                columns5['State'].append(state)
                columns5['Year'].append(year)
                columns5['Quarter'].append(int(file.strip('.json')))
df_top_trans = pd.DataFrame(columns5)
df_top_trans

,State,Year,Quarter,Pincode,Transaction_count,Transaction_amount
0,maharashtra,2020,1,411004,43278566,6.489494e+10
1,maharashtra,2020,1,400070,10817537,2.106684e+10
2,maharashtra,2020,1,411057,5637564,3.498482e+09
3,maharashtra,2020,1,411014,3577288,2.709271e+09
4,maharashtra,2020,1,411033,3338361,2.293643e+09
...,...,...,...,...,...,...
8919,chandigarh,2019,4,160047,103726,2.063567e+08
8920,chandigarh,2019,4,160015,70113,1.067267e+08
8921,chandigarh,2019,4,160014,69754,1.114074e+08
8922,chandigarh,2019,4,160019,52557,1.019240e+08


In [16]:
path6 = "/content/pulse/data/top/user/country/india/state/"
top_user_list = os.listdir(path6)
columns6 = {'State': [], 'Year': [], 'Quarter': [], 'Pincode': [],
            'RegisteredUsers': []}

for state in top_user_list:
    cur_state = path6 + state + "/"
    top_year_list = os.listdir(cur_state)

    for year in top_year_list:
        cur_year = cur_state + year + "/"
        top_file_list = os.listdir(cur_year)

        for file in top_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            F = json.load(data)

            for i in F['data']['pincodes']:
                name = i['name']
                registeredUsers = i['registeredUsers']
                columns6['Pincode'].append(name)
                columns6['RegisteredUsers'].append(registeredUsers)
                columns6['State'].append(state)
                columns6['Year'].append(year)
                columns6['Quarter'].append(int(file.strip('.json')))
df_top_user = pd.DataFrame(columns6)
df_top_user

,State,Year,Quarter,Pincode,RegisteredUsers
0,maharashtra,2020,1,421302,270275
1,maharashtra,2020,1,410501,212525
2,maharashtra,2020,1,410206,208988
3,maharashtra,2020,1,421201,201500
4,maharashtra,2020,1,411014,186363
...,...,...,...,...,...
8920,chandigarh,2019,4,160014,19040
8921,chandigarh,2019,4,160011,11692
8922,chandigarh,2019,4,160015,11476
8923,chandigarh,2019,4,160030,10959


In [17]:
!pip install pandas sqlalchemy streamlit plotly ngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.0 MB/s eta 0:00:00


In [26]:
import sqlite3

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('phonepe_pulse.db')
cursor = conn.cursor()

# Create tables in the SQLite database
cursor.execute('''
CREATE TABLE IF NOT EXISTS aggregated_transaction (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    State TEXT,
    Year TEXT,
    Quarter INTEGER,
    Transaction_type TEXT,
    Transaction_count INTEGER,
    Transaction_amount REAL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS aggregated_user (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    State TEXT,
    Year TEXT,
    Quarter INTEGER,
    Brands TEXT,
    Count INTEGER,
    Percentage REAL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS map_transaction (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    State TEXT,
    Year TEXT,
    Quarter INTEGER,
    District TEXT,
    Count INTEGER,
    Amount REAL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS map_user (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    State TEXT,
    Year TEXT,
    Quarter INTEGER,
    District TEXT,
    RegisteredUser INTEGER,
    AppOpens INTEGER
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS top_transaction (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    State TEXT,
    Year TEXT,
    Quarter INTEGER,
    Pincode TEXT,
    Transaction_count INTEGER,
    Transaction_amount REAL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS top_user (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    State TEXT,
    Year TEXT,
    Quarter INTEGER,
    Pincode TEXT,
    RegisteredUsers INTEGER
)
''')

# Insert data into the tables
df_agg_trans.to_sql('aggregated_transaction', conn, if_exists='append', index=False)
df_agg_user.to_sql('aggregated_user', conn, if_exists='append', index=False)
df_map_trans.to_sql('map_transaction', conn, if_exists='append', index=False)
df_map_user.to_sql('map_user', conn, if_exists='append', index=False)
df_top_trans.to_sql('top_transaction', conn, if_exists='append', index=False)
df_top_user.to_sql('top_user', conn, if_exists='append', index=False)

# Commit changes and close the connection
conn.commit()
conn.close()


In [28]:
# Reconnect to the SQLite database
conn = sqlite3.connect('phonepe_pulse.db')
cursor = conn.cursor()


In [29]:
# List all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Print the table names
for table in tables:
    print(table[0])


transaction_aggregated
user_aggregated
transaction_hover_map
user_hover_map
transaction_top
user_top
aggregated_transaction
sqlite_sequence
aggregated_user
map_transaction
map_user
top_transaction
top_user


In [30]:
# Describe a table (e.g., 'aggregated_transaction')
cursor.execute("PRAGMA table_info(aggregated_transaction);")
columns = cursor.fetchall()

# Print the column names and types
for column in columns:
    print(column)


(0, 'id', 'INTEGER', 0, None, 1)
(1, 'State', 'TEXT', 0, None, 0)
(2, 'Year', 'TEXT', 0, None, 0)
(3, 'Quarter', 'INTEGER', 0, None, 0)
(4, 'Transaction_type', 'TEXT', 0, None, 0)
(5, 'Transaction_count', 'INTEGER', 0, None, 0)
(6, 'Transaction_amount', 'REAL', 0, None, 0)


In [32]:
# Check the first few rows of a table (e.g., 'aggregated_transaction')
cursor.execute("SELECT * FROM aggregated_transaction ;")
rows = cursor.fetchall()

# Print the rows
for row in rows:
    print(row)


(1, 'maharashtra', '2020', 1, 'Merchant payments', 100337738, 35200130304.82776)
(2, 'maharashtra', '2020', 1, 'Peer-to-peer payments', 87067410, 262541205695.8384)
(3, 'maharashtra', '2020', 1, 'Recharge & bill payments', 50199498, 21603565070.425602)
(4, 'maharashtra', '2020', 1, 'Financial Services', 123476, 91363684.1276232)
(5, 'maharashtra', '2020', 1, 'Others', 1215959, 558931962.876125)
(6, 'maharashtra', '2020', 2, 'Peer-to-peer payments', 74295930, 239933338682.6934)
(7, 'maharashtra', '2020', 2, 'Recharge & bill payments', 60856480, 23112343247.416145)
(8, 'maharashtra', '2020', 2, 'Merchant payments', 48965388, 32859184582.219048)
(9, 'maharashtra', '2020', 2, 'Financial Services', 206461, 211733275.7829895)
(10, 'maharashtra', '2020', 2, 'Others', 466502, 242917371.3033281)
(11, 'maharashtra', '2020', 3, 'Peer-to-peer payments', 108436214, 351555554813.2122)
(12, 'maharashtra', '2020', 3, 'Merchant payments', 100808237, 60171102123.94888)
(13, 'maharashtra', '2020', 3, 'Re

In [33]:
# Close the connection to the database
conn.close()


In [34]:
!pip install streamlit pyngrok plotly pandas sqlite3


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [62]:
%%writefile app.py
import streamlit as st
import plotly.express as px
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('phonepe_pulse.db')

# Function to load data from a table
def load_data(table_name):
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql(query, conn)
    return df

# Streamlit App
st.title("PhonePe Pulse Geo-Visualization Dashboard")

# Sidebar options for selecting different visualizations
st.sidebar.title("Select Visualization")
options = st.sidebar.selectbox(
    "Choose a dataset to visualize",
    ("Aggregated Transaction Data",
     "Aggregated User Data",
     "Map Transaction Data",
     "Map User Data",
     "Top Transaction Data",
     "Top User Data")
)

# Dropdown options to filter data and create visualizations
if options == "Map Transaction Data":
    df = load_data('map_transaction')
    state = st.sidebar.selectbox("Select State", df['State'].unique())
    year = st.sidebar.selectbox("Select Year", df['Year'].unique())
    quarter = st.sidebar.selectbox("Select Quarter", df['Quarter'].unique())

    filtered_df = df[(df['State'] == state) & (df['Year'] == year) & (df['Quarter'] == quarter)]

    if not filtered_df.empty:
        fig = px.choropleth(
            filtered_df,
            geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
            featureidkey="properties.ST_NM",
            locations="State",
            color="Amount",
            color_continuous_scale="Blues",
            title=f"Transaction Amount by State in {state} ({year} Q{quarter})"
        )

        # Update GeoJSON settings
        fig.update_geos(fitbounds="locations", visible=False)
        st.plotly_chart(fig)
    else:
        st.write("No data available for the selected filters.")

elif options == "Map User Data":
    df = load_data('map_user')
    state = st.sidebar.selectbox("Select State", df['State'].unique())
    year = st.sidebar.selectbox("Select Year", df['Year'].unique())
    quarter = st.sidebar.selectbox("Select Quarter", df['Quarter'].unique())

    filtered_df = df[(df['State'] == state) & (df['Year'] == year) & (df['Quarter'] == quarter)]

    if not filtered_df.empty:
        fig = px.choropleth(
            filtered_df,
            geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
            featureidkey="properties.ST_NM",
            locations="State",
            color="RegisteredUser",
            color_continuous_scale="Blues",
            title=f"Registered Users by State in {state} ({year} Q{quarter})"
        )

        # Update GeoJSON settings
        fig.update_geos(fitbounds="locations", visible=False)
        st.plotly_chart(fig)
    else:
        st.write("No data available for the selected filters.")

elif options == "Aggregated Transaction Data":
    df = load_data('aggregated_transaction')
    state = st.sidebar.selectbox("Select State", df['State'].unique())
    year = st.sidebar.selectbox("Select Year", df['Year'].unique())
    quarter = st.sidebar.selectbox("Select Quarter", df['Quarter'].unique())

    filtered_df = df[(df['State'] == state) & (df['Year'] == year) & (df['Quarter'] == quarter)]

    if not filtered_df.empty:
        fig = px.bar(
            filtered_df,
            x='Transaction_type',
            y='Transaction_amount',
            color='Transaction_type',
            title=f"Transaction Amount by Type in {state} ({year} Q{quarter})"
        )
        st.plotly_chart(fig)
    else:
        st.write("No data available for the selected filters.")

elif options == "Aggregated User Data":
    df = load_data('aggregated_user')
    state = st.sidebar.selectbox("Select State", df['State'].unique())
    year = st.sidebar.selectbox("Select Year", df['Year'].unique())
    quarter = st.sidebar.selectbox("Select Quarter", df['Quarter'].unique())

    filtered_df = df[(df['State'] == state) & (df['Year'] == year) & (df['Quarter'] == quarter)]

    if not filtered_df.empty:
        fig = px.pie(
            filtered_df,
            values='Count',
            names='Brands',
            title=f"User Device Brands Distribution in {state} ({year} Q{quarter})"
        )
        st.plotly_chart(fig)
    else:
        st.write("No data available for the selected filters.")

elif options == "Top Transaction Data":
    df = load_data('top_transaction')
    state = st.sidebar.selectbox("Select State", df['State'].unique())
    year = st.sidebar.selectbox("Select Year", df['Year'].unique())
    quarter = st.sidebar.selectbox("Select Quarter", df['Quarter'].unique())

    filtered_df = df[(df['State'] == state) & (df['Year'] == year) & (df['Quarter'] == quarter)]

    if not filtered_df.empty:
        fig = px.bar(
            filtered_df,
            x='Pincode',
            y='Transaction_amount',
            color='Transaction_amount',
            title=f"Transaction Amount by Pincode in {state} ({year} Q{quarter})"
        )
        st.plotly_chart(fig)
    else:
        st.write("No data available for the selected filters.")

elif options == "Top User Data":
    df = load_data('top_user')
    state = st.sidebar.selectbox("Select State", df['State'].unique())
    year = st.sidebar.selectbox("Select Year", df['Year'].unique())
    quarter = st.sidebar.selectbox("Select Quarter", df['Quarter'].unique())

    filtered_df = df[(df['State'] == state) & (df['Year'] == year) & (df['Quarter'] == quarter)]

    if not filtered_df.empty:
        fig = px.bar(
            filtered_df,
            x='Pincode',
            y='RegisteredUsers',
            color='RegisteredUsers',
            title=f"Registered Users by Pincode in {state} ({year} Q{quarter})"
        )
        st.plotly_chart(fig)
    else:
        st.write("No data available for the selected filters.")

# Closing the database connection
conn.close()


Overwriting app.py


In [64]:
import pandas as pd
import plotly.express as px

# Load data
df = pd.read_csv("https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/active_cases_2020-07-17_0800.csv")

# Rename 'active cases' to 'transactions'
df.rename(columns={'active cases': 'transactions'}, inplace=True)

# Print column names to verify
print(df.columns)

# Check if 'transactions' column exists
if 'transactions' in df.columns:
    # Create choropleth map
    fig = px.choropleth(
        df,
        geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
        featureidkey='properties.ST_NM',
        locations='state',                # Column in DataFrame to match with GeoJSON
        color='transactions',             # Column in DataFrame to visualize
        color_continuous_scale='Blues'    # Predefined color scale
    )

    # Update GeoJSON settings
    fig.update_geos(fitbounds="locations", visible=False)

    # Show the figure
    fig.show()

else:
    print("The column you are trying to use for the color does not exist in the DataFrame.")


Index(['state', 'transactions'], dtype='object')


In [36]:
import os
os.environ["NGROK_AUTH_TOKEN"] = "2iT1S8jORWP8AFT2gipGnOs9RQe_325YotmHc3irE4pTUSuSp"

In [37]:
# Install ngrok
!pip install pyngrok

  Using cached pyngrok-7.2.0-py3-none-any.whl.metadata (7.4 kB)


In [47]:
import pandas as pd
import plotly.express as px

df = pd.read_csv("https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/active_cases_2020-07-17_0800.csv")

# Example filtering: Keep rows where 'active cases' are greater than 1000
filtered_df = df[df['active cases'] > 1000]

print(filtered_df.columns)

Index(['state', 'active cases'], dtype='object')


In [50]:
def load_data(table_name):
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql(query, conn)

    # Print column names to verify
    print(f"Columns in {table_name}: {df.columns.tolist()}")

    # Rename 'active cases' to 'transactions' if it exists
    if 'active cases' in df.columns:
        df.rename(columns={'active cases': 'transactions'}, inplace=True)

    return df


In [53]:
import pandas as pd
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('phonepe_pulse.db')

def load_data(table_name):
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql(query, conn)

    # Print column names to verify
    print(f"Columns in {table_name}: {df.columns.tolist()}")

    # Rename columns as needed
    df.rename(columns={
        'active cases': 'transactions',
        'Transaction_amount': 'transactions',
        'Latitude': 'lat',
        'Longitude': 'lon'
    }, inplace=True)

    return df

# Test loading and renaming columns
df_test = load_data('map_transaction')
print(df_test.head())


Columns in map_transaction: ['id', 'State', 'Year', 'Quarter', 'District', 'Count', 'Amount']
   id        State  Year  Quarter            District     Count        Amount
0   1  maharashtra  2020        1    jalgaon district   2925407  4.531030e+09
1   2  maharashtra  2020        1      latur district   2119578  3.427202e+09
2   3  maharashtra  2020        1      thane district  15699451  2.255105e+10
3   4  maharashtra  2020        1   bhandara district    687372  1.008982e+09
4   5  maharashtra  2020        1  nandurbar district    509151  9.110139e+08


In [63]:
# Import ngrok
from pyngrok import ngrok

# Authenticate ngrok with your token
ngrok.set_auth_token("2iT1S8jORWP8AFT2gipGnOs9RQe_325YotmHc3irE4pTUSuSp")

# Start ngrok to tunnel the Streamlit app
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")

# Run the Streamlit app
!streamlit run app.py &


Streamlit app is live at: NgrokTunnel: "https://2cee-34-83-89-151.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.83.89.151:8501

  Stopping...
